# Modeles

https://www.kaggle.com/c/home-credit-default-risk

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd

from hyperopt import hp, tpe
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

import xgboost as xgb

import lightgbm as lgbm


import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import gc

# bokeh basics

#from bokeh.plotting import figure
#from bokeh.io import show, output_notebook

pd.set_option('display.max_columns', 200) #Pour avoir accès à toutes les colonnes dans la méthode display
pd.set_option('display.max_rows', 200) #Pour avoir accès à toutes les colonnes dans la méthode display
pd.set_option('max_colwidth', 255) 

In [3]:
def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')

In [4]:
# Read train/test table
train = pd.read_csv('../m_train.csv', index_col = 'SK_ID_CURR')

In [5]:
train

,CREDIT_TERM,DAYS_BIRTH,DAYS_EMPLOYED,client_installments_AMT_PAYMENT_min_sum,AMT_ANNUITY,DAYS_LAST_PHONE_CHANGE,bureau_DAYS_CREDIT_max,DAYS_ID_PUBLISH,client_cash_CNT_INSTALMENT_FUTURE_mean_max,AMT_CREDIT,bureau_DAYS_CREDIT_ENDDATE_max,ANNUITY_INCOME_PERCENT,client_installments_AMT_PAYMENT_min_min,bureau_AMT_CREDIT_SUM_DEBT_mean,DAYS_REGISTRATION,bureau_DAYS_CREDIT_mean,CODE_GENDER_F,previous_CNT_PAYMENT_mean,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,bureau_AMT_CREDIT_SUM_mean,client_cash_CNT_INSTALMENT_FUTURE_mean_mean,bureau_AMT_CREDIT_MAX_OVERDUE_sum,previous_HOUR_APPR_PROCESS_START_mean,client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_min,client_installments_AMT_PAYMENT_sum_mean,previous_AMT_DOWN_PAYMENT_mean,bureau_AMT_CREDIT_SUM_min,client_installments_AMT_PAYMENT_min_max,bureau_AMT_CREDIT_SUM_max,CREDIT_INCOME_PERCENT,previous_NAME_YIELD_GROUP_low_action_mean,previous_RATE_DOWN_PAYMENT_max,client_cash_CNT_INSTALMENT_FUTURE_min_max,bureau_AMT_CREDIT_SUM_LIMIT_mean,bureau_DAYS_CREDIT_UPDATE_max,client_installments_NUM_INSTALMENT_VERSION_mean_max,previous_DAYS_DECISION_max,client_installments_DAYS_ENTRY_PAYMENT_sum_max,client_installments_AMT_PAYMENT_sum_max,bureau_CREDIT_ACTIVE_Active_sum,previous_SELLERPLACE_AREA_max,client_installments_NUM_INSTALMENT_VERSION_mean_mean,bureau_AMT_CREDIT_MAX_OVERDUE_mean,previous_PRODUCT_COMBINATION_CashXSelllow_mean,client_credit_CNT_DRAWINGS_CURRENT_mean_mean,OWN_CAR_AGE,NAME_EDUCATION_TYPE_Highereducation,bureau_DAYS_CREDIT_ENDDATE_mean,previous_NAME_CONTRACT_STATUS_Refused_mean,bureau_DAYS_ENDDATE_FACT_max,NAME_FAMILY_STATUS_Married,client_installments_AMT_PAYMENT_mean_max,bureau_DAYS_CREDIT_sum,previous_NAME_YIELD_GROUP_high_mean,previous_AMT_ANNUITY_min,previous_AMT_ANNUITY_max,client_installments_AMT_INSTALMENT_max_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_sum,previous_DAYS_LAST_DUE_1ST_VERSION_sum,client_installments_DAYS_ENTRY_PAYMENT_min_mean,previous_AMT_GOODS_PRICE_min,bureau_CREDIT_TYPE_Mortgage_mean,previous_AMT_ANNUITY_mean,client_installments_AMT_INSTALMENT_max_min,bureau_DAYS_CREDIT_ENDDATE_min,previous_NAME_TYPE_SUITE_Unaccompanied_mean,previous_RATE_DOWN_PAYMENT_sum,client_installments_AMT_PAYMENT_sum_min,client_installments_AMT_PAYMENT_mean_mean,previous_DAYS_FIRST_DUE_mean,client_installments_AMT_INSTALMENT_min_sum,bureau_AMT_CREDIT_SUM_DEBT_max,previous_AMT_CREDIT_max,previous_AMT_CREDIT_mean,AMT_INCOME_TOTAL,previous_NAME_YIELD_GROUP_low_normal_mean,previous_HOUR_APPR_PROCESS_START_max,bureau_AMT_CREDIT_SUM_OVERDUE_mean,client_credit_CNT_DRAWINGS_CURRENT_max_min,bureau_DAYS_CREDIT_UPDATE_mean,client_cash_NAME_CONTRACT_STATUS_Active_sum_mean,client_cash_MONTHS_BALANCE_min_mean,client_installments_AMT_INSTALMENT_max_sum,client_cash_CNT_INSTALMENT_FUTURE_mean_min,previous_CHANNEL_TYPE_Creditandcashoffices_mean,bureau_CREDIT_ACTIVE_Closed_mean,client_cash_CNT_INSTALMENT_FUTURE_min_mean,previous_RATE_DOWN_PAYMENT_mean,previous_AMT_ANNUITY_sum,bureau_AMT_CREDIT_SUM_LIMIT_sum,previous_CNT_PAYMENT_max,client_cash_MONTHS_BALANCE_sum_max,client_cash_MONTHS_BALANCE_sum_min,FLAG_DOCUMENT_3,previous_DAYS_LAST_DUE_max,client_cash_CNT_INSTALMENT_FUTURE_mean_sum,client_cash_MONTHS_BALANCE_min_max,previous_DAYS_DECISION_mean,...,previous_NAME_SELLER_INDUSTRY_Consumerelectronics_mean,client_cash_NAME_CONTRACT_STATUS_Signed_mean_mean,client_credit_CNT_DRAWINGS_POS_CURRENT_max_sum,client_bureau_balance_STATUS_X_sum_mean,client_credit_AMT_DRAWINGS_POS_CURRENT_sum_sum,client_bureau_balance_MONTHS_BALANCE_max_sum,previous_NAME_CONTRACT_TYPE_Revolvingloans_mean,AMT_REQ_CREDIT_BUREAU_MON,OCCUPATION_TYPE_Highskilltechstaff,client_bureau_balance_MONTHS_BALANCE_max_mean,ORGANIZATION_TYPE_Military,previous_NAME_PORTFOLIO_XNA_mean,ORGANIZATION_TYPE_Industrytype9,previous_PRODUCT_COMBINATION_POShouseholdwithinterest_mean,client_cash_NAME_CONTRACT_STATUS_Active_sum_min,client_cash_NAME_CONTRACT_STATUS_Completed_mean_mean,previous_NAME_SELLER_INDUSTRY_Clothing_mean,client_cash_NAME_CO

# LightGBM

## Créer un scorer 
- Imaginons que l'organisme de crédit perde 10.000 par crédit remboursé "manqué" (faux positif)
- Qu'il perde 100.000 par crédit accepté non remboursé (faux négatif).

In [6]:
def scoring_fcn(y_true, y_pred, cout_FN = 100000, cout_FP = 10000):
    # Cout des faux negatifs: 
    loss_function = cout_FN* sum((1-y_true)*(y_pred))
    # Cout des faux positifs
    loss_function += cout_FP* sum((y_true)*(1-y_pred))
    
    return loss_function/len(y_true)

- Créer un autre sous-ensemble équilibré en TARGET
- Faire tourner le modèle avec les hyperparamètres trouvés précédemment, sur le sous-ensemble
- Récupérer le modèle pour pouvoir l'appeler ensuite avec une api

In [7]:
y_true = np.array([0,0,1,1,0,1])
y_pred = np.array([1,0,1,0,1,1])

scoring_fcn(y_true, y_pred)

35000.0

In [8]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=8000, random_state=42)).reset_index(drop = True)

In [9]:
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']

In [10]:
y.value_counts()

False    8000
True     8000
Name: TARGET, dtype: int64

In [11]:
def objective_weighted(params):
    params = {'boosting_type' : params['boosting_type'],
        'num_leaves': int(params['num_leaves']),
              'colsample_bytree': '{:.3f}'.format(params['colsample_bytree'])}
    clf = lgbm.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        **params)
    
    # L'objectif est de minimiser la "loss fonction" définie par la fonction scoring_fcn
    
    
    from sklearn.metrics import make_scorer
    
    score = cross_val_score(clf, X, y, scoring=make_scorer(scoring_fcn), cv=10).mean()
    print("loss_function {:.3f} params {}".format(score, params))
    return score


space = {
    'boosting_type' : hp.choice('boosting_type', ('gbdt','dart','goss')),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}


best = fmin(fn=objective_weighted,
            space=space,
            algo=tpe.suggest,
            max_evals=30)

loss_function 17705.000 params {'boosting_type': 'goss', 'num_leaves': 68, 'colsample_bytree': '0.867'}
loss_function 17436.250 params {'boosting_type': 'goss', 'num_leaves': 110, 'colsample_bytree': '0.382'}
loss_function 17508.750 params {'boosting_type': 'gbdt', 'num_leaves': 24, 'colsample_bytree': '0.571'}
loss_function 18142.500 params {'boosting_type': 'dart', 'num_leaves': 60, 'colsample_bytree': '0.890'}
loss_function 17397.500 params {'boosting_type': 'gbdt', 'num_leaves': 64, 'colsample_bytree': '0.301'}
loss_function 17618.750 params {'boosting_type': 'goss', 'num_leaves': 36, 'colsample_bytree': '0.388'}
loss_function 17513.125 params {'boosting_type': 'goss', 'num_leaves': 64, 'colsample_bytree': '0.442'}
loss_function 18337.500 params {'boosting_type': 'dart', 'num_leaves': 36, 'colsample_bytree': '0.756'}
loss_function 17603.750 params {'boosting_type': 'goss', 'num_leaves': 124, 'colsample_bytree': '0.468'}
loss_function 18361.875 params {'boosting_type': 'dart', 'num_

In [12]:
best

{'boosting_type': 0,
 'colsample_bytree': 0.3560992793108581,
 'num_leaves': 110.0}

## Entrainer le scorer sur un plus gros set, pour l'utiliser ensuite

In [54]:
train['TARGET'].value_counts()

False    282686
True      24825
Name: TARGET, dtype: int64

In [55]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=24000, random_state=42)).reset_index(drop = True)
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']
y.value_counts()

False    24000
True     24000
Name: TARGET, dtype: int64

In [56]:
clf_final_lgbm = lgbm.LGBMClassifier(n_estimators=500, learning_rate=0.01, boosting_type = 'gbdt',
 colsample_bytree = 0.3560992793108581,
 num_leaves = 110, random_state = 42)
clf_final_lgbm.fit(X,y)

LGBMClassifier(colsample_bytree=0.3560992793108581, learning_rate=0.01,
               n_estimators=500, num_leaves=110, random_state=42)

In [57]:
y_pred_lgbm = clf_final_lgbm.predict(X)
scoring_fcn(y, y_pred_lgbm)

10702.708333333334

In [58]:
clf_final_lgbm

LGBMClassifier(colsample_bytree=0.3560992793108581, learning_rate=0.01,
               n_estimators=500, num_leaves=110, random_state=42)

In [59]:
# Save classifier
import pickle
# save the model to disk
filename = '../app/models/finalized_model_lgbm.sav'
pickle.dump(clf_final_lgbm, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [60]:
test = pd.read_csv('../app/data/m_test.csv', index_col = 'SK_ID_CURR')
Y_TEST_PREDICT = clf_final_lgbm.predict(test)

In [62]:
Y_TEST_PROBA = clf_final_lgbm.predict_proba(test)

In [65]:
max(Y_TEST_PROBA[:,1])

0.918064161136976

In [66]:
np.where(Y_TEST_PROBA[:,1]==max(Y_TEST_PROBA[:,1]))

(array([32608]),)

In [67]:
test.iloc[32608]

CREDIT_TERM                                                    20.0
DAYS_BIRTH                                                    -8528
DAYS_EMPLOYED                                                -279.0
client_installments_AMT_PAYMENT_min_sum                     3298.32
AMT_ANNUITY                                                 19449.0
                                                             ...   
previous_NAME_CLIENT_TYPE_Repeater_mean                    0.666667
previous_NAME_TYPE_SUITE_Unaccompanied_sum                      1.0
client_credit_CNT_DRAWINGS_POS_CURRENT_sum_min                  NaN
client_bureau_balance_STATUS_1_sum_mean                         0.5
previous_NAME_GOODS_CATEGORY_ConstructionMaterials_mean         0.0
Name: 337338, Length: 339, dtype: object

In [28]:
np.unique(Y_TEST_PREDICT, return_counts = True)

(array([False,  True]), array([34510, 14234]))

# Random Forest

In [29]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=8000, random_state=42)).reset_index(drop = True)

In [30]:
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']

In [31]:
y.value_counts()

False    8000
True     8000
Name: TARGET, dtype: int64

In [33]:
def objective_weighted(params):
    
    from sklearn.ensemble import RandomForestClassifier
    
    params = {'n_estimators' : int(params['n_estimators']),
        'max_depth': int(params['max_depth']),
              'min_samples_leaf': float(params['min_samples_leaf'])}
    clf = RandomForestClassifier(n_estimators = 500, max_depth = 50, criterion='gini', min_samples_leaf=0.01)
    # L'objectif est de minimiser la "loss fonction" définie par la fonction scoring_fcn
    
    
    from sklearn.metrics import make_scorer
    from sklearn.impute import SimpleImputer
    
    # Create our imputer to replace missing values with the mean e.g.
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp = imp.fit(X)

    # Impute our data, then train
    X_train_imp = imp.transform(X)
    
    
    
    score = cross_val_score(clf, X_train_imp, y, scoring=make_scorer(scoring_fcn), cv=10).mean()
    print("loss_function {:.3f} params {}".format(score, params))
    return score


space = {
    'n_estimators' : hp.uniform('n_estimators', 100, 500),
    'max_depth': hp.uniform('max_depth', 10, 50),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 0.01, 0.1),
}


best_rf = fmin(fn=objective_weighted,
            space=space,
            algo=tpe.suggest,
            max_evals=30)

loss_function 18819.375 params {'n_estimators': 436, 'max_depth': 25, 'min_samples_leaf': 0.05099066626820067}
loss_function 18714.375 params {'n_estimators': 154, 'max_depth': 21, 'min_samples_leaf': 0.07712947260998261}
loss_function 18825.000 params {'n_estimators': 208, 'max_depth': 48, 'min_samples_leaf': 0.047310512000999116}
loss_function 18922.500 params {'n_estimators': 479, 'max_depth': 17, 'min_samples_leaf': 0.08081644157173073}
loss_function 18917.500 params {'n_estimators': 150, 'max_depth': 19, 'min_samples_leaf': 0.055088860021058485}
loss_function 18938.125 params {'n_estimators': 143, 'max_depth': 21, 'min_samples_leaf': 0.08390504924319715}
loss_function 18976.250 params {'n_estimators': 290, 'max_depth': 39, 'min_samples_leaf': 0.06799831753814088}
loss_function 18780.000 params {'n_estimators': 292, 'max_depth': 49, 'min_samples_leaf': 0.09267955567916485}
loss_function 18896.250 params {'n_estimators': 149, 'max_depth': 28, 'min_samples_leaf': 0.010870161062464639

In [34]:
best

{'max_depth': 21.989896116337036,
 'min_samples_leaf': 0.03657344445087089,
 'n_estimators': 208.85926188025218}

## Entrainer le scorer sur un plus gros set, pour l'utiliser ensuite

In [35]:
train['TARGET'].value_counts()

False    282686
True      24825
Name: TARGET, dtype: int64

In [36]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=24000, random_state=42)).reset_index(drop = True)
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']
y.value_counts()

False    24000
True     24000
Name: TARGET, dtype: int64

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X)

# Impute our data, then train
X_train_imp = imp.transform(X)


clf_final_rf = RandomForestClassifier(max_depth = 22, min_samples_leaf = 0.03657344445087089, n_estimators = 210,
                                     random_state = 42)
clf_final_rf.fit(X_train_imp,y)

RandomForestClassifier(max_depth=22, min_samples_leaf=0.03657344445087089,
                       n_estimators=210)

In [40]:
y_pred_rf = clf_final_rf.predict(X_train_imp)
scoring_fcn(y, y_pred_rf)

19211.875

In [41]:
clf_final_rf

RandomForestClassifier(max_depth=22, min_samples_leaf=0.03657344445087089,
                       n_estimators=210)

In [43]:
# Save classifier
import pickle
# save the model to disk
filename = '../app/models/finalized_model_rf.sav'
pickle.dump(clf_final_rf, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [45]:
test = pd.read_csv('../app/data/m_test.csv', index_col = 'SK_ID_CURR')
# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(test)

# Impute our data, then train
X_test_imp = imp.transform(test)
Y_TEST_PREDICT = clf_final_rf.predict(X_test_imp)

In [46]:
np.unique(Y_TEST_PREDICT, return_counts = True)

(array([False,  True]), array([31382, 17362]))

# SVM

In [81]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=8000, random_state=42)).reset_index(drop = True)

In [82]:
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']

In [83]:
y.value_counts()

False    8000
True     8000
Name: TARGET, dtype: int64

In [84]:
def objective_weighted(params):
    
    from sklearn.svm import LinearSVC
    from sklearn.preprocessing import StandardScaler

    
    params = {'C' : float(params['C'])}
    clf = LinearSVC(C=0.1, random_state = 42, max_iter=10000, dual=False)
    # L'objectif est de minimiser la "loss fonction" définie par la fonction scoring_fcn
    
    
    from sklearn.metrics import make_scorer
    from sklearn.impute import SimpleImputer
    
    # Create our imputer to replace missing values with the mean e.g.
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp = imp.fit(X)

    # Impute our data, then train
    X_train_imp = imp.transform(X)
    
    # Standardize data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train_imp)
    
    score = cross_val_score(clf, X_train_scale, y, scoring=make_scorer(scoring_fcn), cv=10).mean()
    print("loss_function {:.3f} params {}".format(score, params))
    return score


space = {
    'C' : hp.uniform('C', 0.1, 1)}


best_svm = fmin(fn=objective_weighted,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

loss_function 18302.500 params {'C': 0.6838334452216949}
loss_function 18302.500 params {'C': 0.23869731479407208}            
loss_function 18302.500 params {'C': 0.581670403384514}              
loss_function 18302.500 params {'C': 0.6383690513956735}             
loss_function 18302.500 params {'C': 0.2798044621578304}             
loss_function 18302.500 params {'C': 0.3440815676736435}             
loss_function 18302.500 params {'C': 0.8414715700286014}             
loss_function 18302.500 params {'C': 0.6626046722882453}             
loss_function 18302.500 params {'C': 0.6325309244039472}             
loss_function 18302.500 params {'C': 0.4115479640701276}             
100%|██████████| 10/10 [12:18<00:00, 73.88s/trial, best loss: 18302.5]


In [85]:
best_svm

{'C': 0.6838334452216949}

## Entrainer le scorer sur un plus gros set, pour l'utiliser ensuite

In [103]:
train['TARGET'].value_counts()

False    282686
True      24825
Name: TARGET, dtype: int64

In [104]:
# pour créer un sous ensemble équilibré en TARGET
train_equilibre = train.groupby('TARGET').apply(lambda x: x.sample(n=24000, random_state=42)).reset_index(drop = True)
X = train_equilibre.drop(['TARGET'], axis=1)
y = train_equilibre['TARGET']
y.value_counts()

False    24000
True     24000
Name: TARGET, dtype: int64

In [105]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X)

# Impute our data, then train
X_train_imp = imp.transform(X)

# Standardize data
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_imp)

print('début de la modélisation')
clf_final_svc = LinearSVC(C = 1, random_state = 42, max_iter=10000, dual=False)

clf_final_svc.fit(X_train_scale,y)

début de la modélisation


LinearSVC(C=1, dual=False, max_iter=10000, random_state=42)

In [106]:
y_pred_svc = clf_final_svc.predict(X_train_scale)
scoring_fcn(y, y_pred_svc)

17726.458333333332

In [107]:
clf_final_svc

LinearSVC(C=1, dual=False, max_iter=10000, random_state=42)

In [108]:
# Save classifier
import pickle
# save the model to disk
filename = '../app/models/finalized_model_svc.sav'
pickle.dump(clf_final_svc, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

In [109]:
test = pd.read_csv('../app/data/m_test.csv', index_col = 'SK_ID_CURR')

test_imp = imp.transform(test)
test_scale = scaler.transform(test_imp)


Y_TEST_PREDICT = clf_final_svc.predict(test_scale)

In [110]:
np.unique(Y_TEST_PREDICT, return_counts = True)

(array([False,  True]), array([33116, 15628]))

# Feature importance

In [ ]:
def identify_zero_importance_features(train, train_labels, iterations = 2):
    """
    Identify zero importance features in a training dataset based on the 
    feature importances from a gradient boosting model. 
    
    Parameters
    --------
    train : dataframe
        Training features
        
    train_labels : np.array
        Labels for training data
        
    iterations : integer, default = 2
        Number of cross validation splits to use for determining feature importances
    """
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    # Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    
    # Fit the model multiple times to avoid overfitting
    for i in range(iterations):

        # Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
    
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances

In [ ]:
feature_importances = pd.DataFrame({'feature': list(X.columns), 
                                    'importance': clf_final.feature_importances_}).sort_values('importance',
                                                                                               ascending = False)
feature_importances['relative_importance'] = 100*feature_importances['importance']/sum(feature_importances['importance'])
feature_importances.head(10)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,6))
sns.barplot(x='relative_importance', y='feature', data = feature_importances.head(10))
plt.xlabel('relative importance (in %)');

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
plt.figure(figsize=(10,8))
sns.barplot(x='importance', y='feature', data = feature_importances.head(20));

In [ ]:
feature_importances['relative_importance']

# AUC-ROC pour Light GBM

In [ ]:
def objective(params):
    params = {'boosting_type' : params['boosting_type'],
        'num_leaves': int(params['num_leaves']),
              'colsample_bytree': '{:.3f}'.format(params['colsample_bytree'])}
    clf = lgbm.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.01,
        **params)
    
    # L'objectif est de maximiser AUC - pour hyperopt on veut minimiser le score
    # d'où le 1-ROCAUC
    score = 1 - cross_val_score(clf, X, y, scoring='roc_auc', cv=10).mean()
    print("ROC-AUC {:.3f} params {}".format(score, params))
    return score


space = {
    'boosting_type' : hp.choice('boosting_type', ('gbdt','dart','goss')),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
}


best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30)